In [3]:
# Imports
import boto3
import json
import numpy as np
import pandas as pd
import time
import datetime

In [4]:
# Configure the SDK to Personalize:
personalize = boto3.client('personalize')
personalize_runtime = boto3.client('personalize-runtime')

In [5]:
# Sets the same region as current Amazon SageMaker Notebook
with open('/opt/ml/metadata/resource-metadata.json') as notebook_info:
    data = json.load(notebook_info)
    resource_arn = data['ResourceArn']
    region = resource_arn.split(':')[3]
print('region:', region)

# Or you can specify the region where your bucket and model will be domiciled
# region = "us-east-1" 

s3 = boto3.client('s3')
account_id = boto3.client('sts').get_caller_identity().get('Account')
bucket_name = account_id + "-" + region + "-" + "personalize"
print('bucket_name:', bucket_name)

try: 
    if region == "us-east-1":
        s3.create_bucket(Bucket=bucket_name)
    else:
        s3.create_bucket(
            Bucket = bucket_name,
            CreateBucketConfiguration={'LocationConstraint': region}
            )
except s3.exceptions.BucketAlreadyOwnedByYou:
    print("Bucket already exists. Using bucket", bucket_name)

region: ap-south-1
bucket_name: 689478441425-ap-south-1-personalize


In [14]:
df=pd.read_csv('Amazon Personalize general data.csv')

df['item']=df['item'].str.lower()

interactions_data=df[['Membership number','item_id','transaction date','price']]

interactions_data=interactions_data.rename(columns={'Membership number':'USER_ID','item_id':'ITEM_ID',
                                                    'transaction date':'TIMESTAMP','price':'EVENT_VALUE'})

interactions_data['EVENT_TYPE']='TRANSACTION'

interactions_data['TIMESTAMP']=pd.to_datetime(interactions_data['TIMESTAMP']).astype(int) / 10**9

interactions_data['TIMESTAMP']=interactions_data['TIMESTAMP'].astype(int)

interactions_data=interactions_data[['USER_ID','ITEM_ID','TIMESTAMP','EVENT_TYPE','EVENT_VALUE']]

interactions_data.EVENT_TYPE.value_counts()

In [58]:
interactions_data.head()

,USER_ID,ITEM_ID,TIMESTAMP,EVENT_TYPE,EVENT_VALUE
0,203867788688,10108,1667692800,TRANSACTION,102
1,131861001206,10109,1612137600,TRANSACTION,565
2,215800110303,10103,1633737600,TRANSACTION,254
3,51396695047,10105,1611705600,TRANSACTION,53
4,70732021477,10104,1634688000,TRANSACTION,57


In [15]:
interactions_data.to_csv('interactions_data.csv')

- Upload to S3

In [16]:
interactions_file_path = 'interactions_data.csv'
boto3.Session().resource('s3').Bucket(bucket_name).Object(interactions_file_path).upload_file(interactions_file_path)
interactions_s3DataPath = "s3://"+bucket_name+"/"+interactions_file_path

- User data

In [21]:
user_data=df[['Membership number','Age','Nationality','Gender']]

user_data=user_data.rename(columns={'Membership number':'USER_ID'})

user_data.dtypes

user_data.to_csv('user_data.csv')

user_file_path = 'user_data.csv'
boto3.Session().resource('s3').Bucket(bucket_name).Object(user_file_path).upload_file(user_file_path)
interactions_s3DataPath = "s3://"+bucket_name+"/"+user_file_path

In [59]:
user_data.head()

,USER_ID,Age,Nationality,Gender
0,203867788688,63,INDIA,Female
1,131861001206,70,ISRAEL,Male
2,215800110303,18,GERMANY,Male
3,51396695047,56,CUBA,Female
4,70732021477,52,PHILIPPINES,Female


- ITEM DATA

In [23]:
item_data=df[['item_id','price','item']]

item_data=item_data.rename(columns={'item':'CATEGORY_L1'})

In [60]:
item_data.head()

,item_id,price,CATEGORY_L1
0,10108,102,fragrances
1,10109,565,gifts
2,10103,254,toys
3,10105,53,chocolates
4,10104,57,wine


In [24]:
item_data.to_csv('item_data.csv')

In [25]:
item_file_path = 'item_data.csv'
boto3.Session().resource('s3').Bucket(bucket_name).Object(item_file_path).upload_file(item_file_path)
interactions_s3DataPath = "s3://"+bucket_name+"/"+item_file_path

- Set the S3 bucket policy

- Amazon Personalize needs to be able to read the contents of your S3 bucket. So add a bucket policy which allows that.

- Note: Make sure the role you are using to run the code in this notebook has the necessary permissions to modify the S3 bucket policy.


In [26]:
s3 = boto3.client("s3")
policy = {
    "Version": "2012-10-17",
    "Id": "PersonalizeS3BucketAccessPolicy",
    "Statement": [
        {
            "Sid": "PersonalizeS3BucketAccessPolicy",
            "Effect": "Allow",
            "Principal": {
                "Service": "personalize.amazonaws.com"
            },
            "Action": [
                "s3:GetObject",
                "s3:ListBucket",
                "s3:PutObject",
            ],
            "Resource": [
                "arn:aws:s3:::{}".format(bucket_name),
                "arn:aws:s3:::{}/*".format(bucket_name)
            ]
        }
    ]
}

s3.put_bucket_policy(Bucket=bucket_name, Policy=json.dumps(policy))

{'ResponseMetadata': {'RequestId': '48Z2E57WD06QJTV1',
  'HostId': 'Rjix3ezNRGnYxUSQUYy6z+VCK91O7OQHUarvkJjOYzxqJKgDRbyv/o8nzGmlfuvtBoSUZp/LffGTat2ayWmVlQ==',
  'HTTPStatusCode': 204,
  'HTTPHeaders': {'x-amz-id-2': 'Rjix3ezNRGnYxUSQUYy6z+VCK91O7OQHUarvkJjOYzxqJKgDRbyv/o8nzGmlfuvtBoSUZp/LffGTat2ayWmVlQ==',
   'x-amz-request-id': '48Z2E57WD06QJTV1',
   'date': 'Fri, 16 Dec 2022 18:35:19 GMT',
   'server': 'AmazonS3'},
  'RetryAttempts': 0}}

In [27]:
response = personalize.create_dataset_group(
    name='personalize_ecommerce_ds_group_v2',
    domain='ECOMMERCE'
)

dataset_group_arn = response['datasetGroupArn']
print(json.dumps(response, indent=2))

{
  "datasetGroupArn": "arn:aws:personalize:ap-south-1:689478441425:dataset-group/personalize_ecommerce_ds_group_v2",
  "domain": "ECOMMERCE",
  "ResponseMetadata": {
    "RequestId": "b96515d1-d190-4cd9-a72c-4ec956afbb20",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "date": "Fri, 16 Dec 2022 18:35:33 GMT",
      "content-type": "application/x-amz-json-1.1",
      "content-length": "134",
      "connection": "keep-alive",
      "x-amzn-requestid": "b96515d1-d190-4cd9-a72c-4ec956afbb20"
    },
    "RetryAttempts": 0
  }
}


In [28]:
%%time

max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_dataset_group_response = personalize.describe_dataset_group(
        datasetGroupArn = dataset_group_arn
    )
    status = describe_dataset_group_response["datasetGroup"]["status"]
    print("DatasetGroup: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)

DatasetGroup: CREATE PENDING
DatasetGroup: ACTIVE
CPU times: user 18.1 ms, sys: 1.43 ms, total: 19.6 ms
Wall time: 1min


In [31]:
users_schema = schema = {
	"type": "record",
	"name": "Users",
	"namespace": "com.amazonaws.personalize.schema",
	"fields": [
		{
			"name": "USER_ID",
			"type": "string"
		},
		{
			"name": "Age",
			"type": "int"
		},
		{
			"name": "Nationality",
			"type": "string"
		},
		{
			"name": "Gender",
			"type": "string"
		}
	],
	"version": "1.0"
}

create_schema_response = personalize.create_schema(
    name = "personalize-ecommerce-users_group0",
    domain = "ECOMMERCE",
    schema = json.dumps(users_schema)
)

users_schema_arn = create_schema_response['schemaArn']
print(json.dumps(create_schema_response, indent=2))

{
  "schemaArn": "arn:aws:personalize:ap-south-1:689478441425:schema/personalize-ecommerce-users_group0",
  "ResponseMetadata": {
    "RequestId": "26146524-2e07-4446-8494-b790edff8194",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "date": "Fri, 16 Dec 2022 18:36:53 GMT",
      "content-type": "application/x-amz-json-1.1",
      "content-length": "101",
      "connection": "keep-alive",
      "x-amzn-requestid": "26146524-2e07-4446-8494-b790edff8194"
    },
    "RetryAttempts": 0
  }
}


In [32]:
interactions_schema = schema = {
	"type": "record",
	"name": "Interactions",
	"namespace": "com.amazonaws.personalize.schema",
	"fields": [
		{
			"name": "USER_ID",
			"type": "string"
		},
		{
			"name": "ITEM_ID",
			"type": "string"
		},
		{
			"name": "TIMESTAMP",
			"type": "long"
		},
		{
			"name": "EVENT_TYPE",
			"type": "string"
		},
		{
			"name": "EVENT_VALUE",
			"type": "float"
		}
	],
	"version": "1.0"
}

create_schema_response = personalize.create_schema(
    name = "personalize-ecommerce-interatn_group0",
    domain = "ECOMMERCE",
    schema = json.dumps(interactions_schema)
)

interaction_schema_arn = create_schema_response['schemaArn']
print(json.dumps(create_schema_response, indent=2))

{
  "schemaArn": "arn:aws:personalize:ap-south-1:689478441425:schema/personalize-ecommerce-interatn_group0",
  "ResponseMetadata": {
    "RequestId": "5435a237-b238-4260-bfe3-061e0841dc2d",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "date": "Fri, 16 Dec 2022 18:36:57 GMT",
      "content-type": "application/x-amz-json-1.1",
      "content-length": "104",
      "connection": "keep-alive",
      "x-amzn-requestid": "5435a237-b238-4260-bfe3-061e0841dc2d"
    },
    "RetryAttempts": 0
  }
}


In [33]:
item_schema = schema = {
	"type": "record",
	"name": "Items",
	"namespace": "com.amazonaws.personalize.schema",
	"fields": [
		{
			"name": "ITEM_ID",
			"type": "string"
		},
		{
			"name": "PRICE",
			"type": "float"
		},
		{
			"name": "CATEGORY_L1",
 "type": [
 "string"
 ],
 "categorical": True
		}
	],
	"version": "1.0"
}

create_schema_response = personalize.create_schema(
    name = "personalize-ecommerce-item_group0",
    domain = "ECOMMERCE",
    schema = json.dumps(item_schema)
)

item_schema_arn = create_schema_response['schemaArn']
print(json.dumps(create_schema_response, indent=2))

{
  "schemaArn": "arn:aws:personalize:ap-south-1:689478441425:schema/personalize-ecommerce-item_group0",
  "ResponseMetadata": {
    "RequestId": "e68f7f50-f876-4c60-9f1c-18fe5eb09f58",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "date": "Fri, 16 Dec 2022 18:37:01 GMT",
      "content-type": "application/x-amz-json-1.1",
      "content-length": "100",
      "connection": "keep-alive",
      "x-amzn-requestid": "e68f7f50-f876-4c60-9f1c-18fe5eb09f58"
    },
    "RetryAttempts": 0
  }
}


In [34]:
dataset_type = "INTERACTIONS"

create_dataset_response = personalize.create_dataset(
    name = "personalize_ecommerce_demo_interactions",
    datasetType = dataset_type,
    datasetGroupArn = dataset_group_arn,
    schemaArn = interaction_schema_arn
)

interactions_dataset_arn = create_dataset_response['datasetArn']
print(json.dumps(create_dataset_response, indent=2))

{
  "datasetArn": "arn:aws:personalize:ap-south-1:689478441425:dataset/personalize_ecommerce_ds_group_v2/INTERACTIONS",
  "ResponseMetadata": {
    "RequestId": "7ec2d95c-8dde-4dfe-b805-37ac365022fd",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "date": "Fri, 16 Dec 2022 18:37:03 GMT",
      "content-type": "application/x-amz-json-1.1",
      "content-length": "115",
      "connection": "keep-alive",
      "x-amzn-requestid": "7ec2d95c-8dde-4dfe-b805-37ac365022fd"
    },
    "RetryAttempts": 0
  }
}


In [35]:
dataset_type = "USERS"

create_dataset_response = personalize.create_dataset(
    name = "personalize_ecommerce_demo_users1",
    datasetType = dataset_type,
    datasetGroupArn = dataset_group_arn,
    schemaArn = users_schema_arn
)

user_dataset_arn = create_dataset_response['datasetArn']
print(json.dumps(create_dataset_response, indent=2))

{
  "datasetArn": "arn:aws:personalize:ap-south-1:689478441425:dataset/personalize_ecommerce_ds_group_v2/USERS",
  "ResponseMetadata": {
    "RequestId": "e25184d9-25eb-418f-8db1-33f167f448b5",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "date": "Fri, 16 Dec 2022 18:37:04 GMT",
      "content-type": "application/x-amz-json-1.1",
      "content-length": "108",
      "connection": "keep-alive",
      "x-amzn-requestid": "e25184d9-25eb-418f-8db1-33f167f448b5"
    },
    "RetryAttempts": 0
  }
}


In [36]:
dataset_type = "ITEMS"

create_dataset_response = personalize.create_dataset(
    name = "personalize_ecommerce_demo_items",
    datasetType = dataset_type,
    datasetGroupArn = dataset_group_arn,
    schemaArn = item_schema_arn
)

item_dataset_arn = create_dataset_response['datasetArn']
print(json.dumps(create_dataset_response, indent=2))

{
  "datasetArn": "arn:aws:personalize:ap-south-1:689478441425:dataset/personalize_ecommerce_ds_group_v2/ITEMS",
  "ResponseMetadata": {
    "RequestId": "54d43471-0712-418a-88f0-579cbfffb563",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "date": "Fri, 16 Dec 2022 18:37:05 GMT",
      "content-type": "application/x-amz-json-1.1",
      "content-length": "108",
      "connection": "keep-alive",
      "x-amzn-requestid": "54d43471-0712-418a-88f0-579cbfffb563"
    },
    "RetryAttempts": 0
  }
}


In [37]:
iam = boto3.client("iam")

role_name = "PersonalizeRoleEcommerceDemoRecommender"
assume_role_policy_document = {
    "Version": "2012-10-17",
    "Statement": [
        {
          "Effect": "Allow",
          "Principal": {
            "Service": "personalize.amazonaws.com"
          },
          "Action": "sts:AssumeRole"
        }
    ]
}

create_role_response = iam.create_role(
    RoleName = role_name,
    AssumeRolePolicyDocument = json.dumps(assume_role_policy_document)
)

# AmazonPersonalizeFullAccess provides access to any S3 bucket with a name that includes "personalize" or "Personalize" 
# if you would like to use a bucket with a different name, please consider creating and attaching a new policy
# that provides read access to your bucket or attaching the AmazonS3ReadOnlyAccess policy to the role
policy_arn = "arn:aws:iam::aws:policy/service-role/AmazonPersonalizeFullAccess"
iam.attach_role_policy(
    RoleName = role_name,
    PolicyArn = policy_arn
)

# Now add S3 support
iam.attach_role_policy(
    PolicyArn='arn:aws:iam::aws:policy/AmazonS3FullAccess',
    RoleName=role_name
)
time.sleep(60) # wait for a minute to allow IAM role policy attachment to propagate

role_arn = create_role_response["Role"]["Arn"]
print(role_arn)

arn:aws:iam::689478441425:role/PersonalizeRoleEcommerceDemoRecommender


In [91]:
#interactions_dataset_arn='arn:aws:personalize:ap-south-1:428544002405:dataset/personalize_ecommerce_ds_group/INTERACTIONS'

In [38]:
create_interactions_dataset_import_job_response = personalize.create_dataset_import_job(
    jobName = "personalize_ecommerce_demo_interactions_import_3",
    datasetArn = interactions_dataset_arn,
    dataSource = {
        "dataLocation": "s3://{}/{}".format(bucket_name, interactions_file_path)
    },
    roleArn = role_arn
)

dataset_interactions_import_job_arn = create_interactions_dataset_import_job_response['datasetImportJobArn']
print(json.dumps(create_interactions_dataset_import_job_response, indent=2))

{
  "datasetImportJobArn": "arn:aws:personalize:ap-south-1:689478441425:dataset-import-job/personalize_ecommerce_demo_interactions_import_3",
  "ResponseMetadata": {
    "RequestId": "4a62af65-cba8-4d12-ad7a-0c5aae23ddaa",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "date": "Fri, 16 Dec 2022 18:38:07 GMT",
      "content-type": "application/x-amz-json-1.1",
      "content-length": "137",
      "connection": "keep-alive",
      "x-amzn-requestid": "4a62af65-cba8-4d12-ad7a-0c5aae23ddaa"
    },
    "RetryAttempts": 0
  }
}


In [39]:
create_user_dataset_import_job_response = personalize.create_dataset_import_job(
    jobName = "personalize_ecommerce_demo_user_import_3",
    datasetArn = user_dataset_arn,
    dataSource = {
        "dataLocation": "s3://{}/{}".format(bucket_name, user_file_path)
    },
    roleArn = role_arn
)

dataset_user_import_job_arn = create_user_dataset_import_job_response['datasetImportJobArn']
print(json.dumps(create_user_dataset_import_job_response, indent=2))

{
  "datasetImportJobArn": "arn:aws:personalize:ap-south-1:689478441425:dataset-import-job/personalize_ecommerce_demo_user_import_3",
  "ResponseMetadata": {
    "RequestId": "ec3bda57-16a7-4b2e-a450-8ae1c3e9b56b",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "date": "Fri, 16 Dec 2022 18:38:39 GMT",
      "content-type": "application/x-amz-json-1.1",
      "content-length": "129",
      "connection": "keep-alive",
      "x-amzn-requestid": "ec3bda57-16a7-4b2e-a450-8ae1c3e9b56b"
    },
    "RetryAttempts": 0
  }
}


In [40]:
create_item_dataset_import_job_response = personalize.create_dataset_import_job(
    jobName = "personalize_ecommerce_demo_item_import_3",
    datasetArn = item_dataset_arn,
    dataSource = {
        "dataLocation": "s3://{}/{}".format(bucket_name, item_file_path)
    },
    roleArn = role_arn
)

dataset_item_import_job_arn = create_item_dataset_import_job_response['datasetImportJobArn']
print(json.dumps(create_item_dataset_import_job_response, indent=2))

{
  "datasetImportJobArn": "arn:aws:personalize:ap-south-1:689478441425:dataset-import-job/personalize_ecommerce_demo_item_import_3",
  "ResponseMetadata": {
    "RequestId": "1dfce189-bbc9-40af-94a8-05bc3f0ea2ca",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "date": "Fri, 16 Dec 2022 18:38:40 GMT",
      "content-type": "application/x-amz-json-1.1",
      "content-length": "129",
      "connection": "keep-alive",
      "x-amzn-requestid": "1dfce189-bbc9-40af-94a8-05bc3f0ea2ca"
    },
    "RetryAttempts": 0
  }
}


In [41]:
%%time

max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_dataset_import_job_response = personalize.describe_dataset_import_job(
        datasetImportJobArn = dataset_interactions_import_job_arn
    )
    status = describe_dataset_import_job_response["datasetImportJob"]['status']
    print("DatasetImportJob: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)

DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: ACTIVE
CPU times: user 41.9 ms, sys: 6.29 ms, total: 48.2 ms
Wall time: 3min


In [42]:
%%time

max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_dataset_user_import_job_arn = personalize.describe_dataset_import_job(
        datasetImportJobArn = dataset_user_import_job_arn
    )
    status = describe_dataset_import_job_response["datasetImportJob"]['status']
    print("DatasetImportJob: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)

DatasetImportJob: ACTIVE
CPU times: user 423 µs, sys: 3.88 ms, total: 4.31 ms
Wall time: 12.4 ms


In [43]:
%%time

max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_dataset_item_import_job_arn = personalize.describe_dataset_import_job(
        datasetImportJobArn = dataset_item_import_job_arn
    )
    status = describe_dataset_import_job_response["datasetImportJob"]['status']
    print("DatasetImportJob: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)

DatasetImportJob: ACTIVE
CPU times: user 3.53 ms, sys: 0 ns, total: 3.53 ms
Wall time: 18.1 ms


- Choose a recommender use cases

In [44]:
available_recipes = personalize.list_recipes(domain='ECOMMERCE') # See a list of recommenders for the domain. 
display (available_recipes['recipes'])

[{'name': 'aws-ecomm-customers-who-viewed-x-also-viewed',
  'recipeArn': 'arn:aws:personalize:::recipe/aws-ecomm-customers-who-viewed-x-also-viewed',
  'status': 'ACTIVE',
  'creationDateTime': datetime.datetime(2019, 6, 10, 0, 0, tzinfo=tzlocal()),
  'lastUpdatedDateTime': datetime.datetime(2022, 12, 12, 16, 29, 40, 39000, tzinfo=tzlocal()),
  'domain': 'ECOMMERCE'},
 {'name': 'aws-ecomm-frequently-bought-together',
  'recipeArn': 'arn:aws:personalize:::recipe/aws-ecomm-frequently-bought-together',
  'status': 'ACTIVE',
  'creationDateTime': datetime.datetime(2019, 6, 10, 0, 0, tzinfo=tzlocal()),
  'lastUpdatedDateTime': datetime.datetime(2022, 12, 12, 16, 29, 40, 39000, tzinfo=tzlocal()),
  'domain': 'ECOMMERCE'},
 {'name': 'aws-ecomm-popular-items-by-purchases',
  'recipeArn': 'arn:aws:personalize:::recipe/aws-ecomm-popular-items-by-purchases',
  'status': 'ACTIVE',
  'creationDateTime': datetime.datetime(2019, 6, 10, 0, 0, tzinfo=tzlocal()),
  'lastUpdatedDateTime': datetime.dateti

In [46]:
create_recommender_response = personalize.create_recommender(
  name = 'recommended_for_you_demo',
  recipeArn = 'arn:aws:personalize:::recipe/aws-ecomm-recommended-for-you',
  datasetGroupArn = dataset_group_arn
)
recommended_for_you_arn = create_recommender_response["recommenderArn"]
print (json.dumps(create_recommender_response))

{"recommenderArn": "arn:aws:personalize:ap-south-1:689478441425:recommender/recommended_for_you_demo", "ResponseMetadata": {"RequestId": "7a688e85-c3c9-4e3b-866a-83b7bbc2c935", "HTTPStatusCode": 200, "HTTPHeaders": {"date": "Fri, 16 Dec 2022 18:43:05 GMT", "content-type": "application/x-amz-json-1.1", "content-length": "101", "connection": "keep-alive", "x-amzn-requestid": "7a688e85-c3c9-4e3b-866a-83b7bbc2c935"}, "RetryAttempts": 0}}


In [47]:
create_solution_response = personalize.create_solution(name = 'customised-solution',
recipeArn = 'arn:aws:personalize:::recipe/aws-user-personalization',
datasetGroupArn = dataset_group_arn
                                                       
)
solution_arn = create_solution_response['solutionArn']
print('solution_arn: ', solution_arn)

solution_arn:  arn:aws:personalize:ap-south-1:689478441425:solution/customised-solution


- Solution Version Response

In [48]:
create_solution_version_response = personalize.create_solution_version(solutionArn =
solution_arn,
trainingMode='FULL')
new_solution_version_arn = create_solution_version_response['solutionVersionArn']
print('solution_version_arn:', new_solution_version_arn)

solution_version_arn: arn:aws:personalize:ap-south-1:689478441425:solution/customised-solution/fc652314


In [51]:
response = personalize.get_solution_metrics(
    solutionVersionArn = new_solution_version_arn)

print(response['metrics'])

{'coverage': 1.0, 'mean_reciprocal_rank_at_25': 0.4158, 'normalized_discounted_cumulative_gain_at_10': 0.6501, 'normalized_discounted_cumulative_gain_at_25': 0.6557, 'normalized_discounted_cumulative_gain_at_5': 0.6142, 'precision_at_10': 0.1083, 'precision_at_25': 0.0442, 'precision_at_5': 0.194}


- Wait for solution to be in active state. It will take sometime dpending on the dataset

- Create Campaign

In [52]:
create_campaign_response = personalize.create_campaign(
name = 'campaign-1',
solutionVersionArn = new_solution_version_arn,
minProvisionedTPS = 1,
campaignConfig = {"itemExplorationConfig": {"explorationWeight": "0.3",
"explorationItemAgeCutOff": "30"}}
)
campaign_arn = create_campaign_response['campaignArn']
print('campaign_arn:', campaign_arn)

campaign_arn: arn:aws:personalize:ap-south-1:689478441425:campaign/campaign-1


In [56]:
rec_response = personalize_runtime.get_recommendations(campaignArn = campaign_arn,
userId = '1234')
print(rec_response['recommendationId'])

RID-f3-4ffe-ae34-7b1a11d25882-CID-1417b7


In [57]:
rec_response['itemList']

[{'itemId': '10108', 'score': 0.1775267},
 {'itemId': '10102', 'score': 0.150655},
 {'itemId': '10103', 'score': 0.1159182},
 {'itemId': '10105', 'score': 0.1045372},
 {'itemId': '10104', 'score': 0.092336},
 {'itemId': '10110', 'score': 0.0685957},
 {'itemId': '10109', 'score': 0.0609578},
 {'itemId': '10106', 'score': 0.0560191},
 {'itemId': '10111', 'score': 0.045798},
 {'itemId': '10112', 'score': 0.0323401},
 {'itemId': '10107', 'score': 0.0238703},
 {'itemId': '10114', 'score': 0.0198007},
 {'itemId': '10101', 'score': 0.0190845},
 {'itemId': '10115', 'score': 0.0162389},
 {'itemId': '10113', 'score': 0.0161717}]